<a href="https://colab.research.google.com/github/ThrallPraudmur/Transformers/blob/main/L1ga_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подчеркнуть из комментариев:
* Заинтересованность в определённых продуктах
* Сезонность (когда гарантия, участие в конкретном тендере)
* Лизинг (машины, оборудование, сезонность)

Итоговый результат - Краткая справка по клиенту.

https://habr.com/ru/articles/768820/ </br>
https://github.com/pestich/text-classification-with-spacy/blob/main/playbook.ipynb

Контакт с ГД. Сказал есть 3 расчетных счета, больше не надо. Есть овер в Альфе, рефить не хотят. Депозиты не интересуют, так как % в ВТБ выгодней. Сам является Клиентом Совкомбанка, как физик, перезвонить через месяц

* Использование Named Entity Recognition для выделения сущностей таких как "расчётные счета", "овердрафт", "депозиты" и т.д.

```python
bank_products = {
    'расчётные счета': [],
    'овердрафт': False,
    'депозиты': False
}

for ent in doc.ents:
    if 'расчётный счёт' in ent.text:
        bank_products['расчётные счета'].append(ent.text)
    elif 'овердрафт' in ent.text:
        bank_products['овердрафт'].append(ent.text)
...
```
бк с ГД, имеется овердрафт в ВТБ 40 млн руб под 9,8%, реф и вкл не интересует, полностью устраивает овердрафт. По иным продуктам потребности нет, кроме лизинга. В лизинг интересует оборудование, пока конкретную информацию дать не может, тк еще не выбрали, что нужно

```python
overdraft_info = {
    'банк': None,
    'сумма': None,
}

for ent on doc.ents:
    if 'овердрафт' in ent.text:
        overdraft_info['банк'] = ent.text
    elif 'млн руб' in ent.text:
        overdraft_info['сумма'] = ent.text
...
```


```python
import spacy
from gensim.models import KeyedVectors

#загрузка предварительно обученных векторов
word_vectors = KeyedVectors.load_word2vec_format('path_to_load', binary = True)
#загрузка предварительно обученной модели
nlp = spacy.load('ru_core_news_sm')

nlp.vocab.vectors = spacy.vocab.Vectors(data = word.vectors.syn0, keys = word_vectors.index2word)

#сохранение модели с добавленными векторами (по желанию)
nlp.to_disk('path_to_save')
```

Что можно сделать?
* Обучаем на выделение сущностей PRODUCT
* Находим сущности в комментарии и по шаблону-паттернам находим, где он находится, сумму, отношение

TAG | MORPH | description
--- | --- | ---
ROOT | СКАЗУЕМОЕ |
NMOD | NOUN MODIFIER | зависимое от главного существительное слово ((в) альфе овер)
NSUBJ | NOMINAL SUBJECT | подлежащее (овердрафт устраивает, депозиты интересуют)
XCOMP | OPEN CLAUSAL COMPLEMENT| глагол / прилагательное, которое требует завершения фразы
OBL | OBLIQUE NOMINAL | обстоятельство ((в) лизинг интересует)
ADVMOD | ADVERBIAL MODIFIER | модификатор наречия, слово или фраза, которое уточняет значение наречия в предложении
CONJ | CONJUNCTION | союз
OBJ | OBJECT | существительно, на которое направлено действие
ADJ | ADJECTIVE | прилагательное

* Разметка по позитивным действиям и негативным действиям
* **Если сущность в предложении, то выделяем часть предложения до знака препинания**



In [ ]:
%%capture

!pip install spacy
!python -m spacy download ru_core_news_md #ru_core_news_md

import pandas as pd
import spacy
import re
from tqdm import tqdm
from spacy import displacy
from spacy.training import Example
from sklearn.utils import shuffle

nlp = spacy.load('ru_core_news_md')

ner = nlp.get_pipe('ner')
ner.add_label('PROD')

available_entities = nlp.get_pipe('ner').labels
print('Доступные сущности:', available_entities)

In [ ]:
# optimizer = ner.create_optimizer()
# optimizer = nlp.initialize()

# nlp.update([example], drop=0.1, sgd=optimizer, losses=losses)

In [ ]:
def dict_to_form(dictlist):
    formed_list = []
    for elem in dictlist:
        for key, value in elem.items():
            doc = nlp(value)
            elem[key] = extract_main_idea(doc)
            formed_list.append(elem)
    return formed_list

In [ ]:
#ФУНКЦИЯ ИЗВЛЕЧЕНИЯ ГЛАВНОЙ МЫСЛИ
def extract_main_idea(sentence):
    main_idea = []
    for token in sentence:
        if token.dep_ not in ['punct']:
            if token.dep_ == 'ROOT' or token.dep_ == 'conj':
                main_idea.append(token.text)
            elif token.dep_ == 'nsubj' and token.head.dep_ == 'ROOT':
                main_idea.append(token.text)
    return ' '.join(main_idea)

In [ ]:
#СПИСОК СЛОВАРЕЙ -> СПИСОК СЛОВАРЕЙ С ОБРАБОТАННЫМИ КЛЮЧАМИ
data_to_train['main_idea'] = data_to_train['entlist'].apply(dict_to_form)

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

data_to_train[['text', 'label', 'main_idea']]

,text,label,main_idea
0,"кт с бухгалтером, предложение лизинга и кик, инфо просила направить на адрес (направлено), доведт до руководителя , созвонимся позже (гд в отъезде)!","[(лизинга, PROD), (кик, PROD)]","[{'лизинга': 'предложение и'}, {'кик': 'предложение и'}]"
1,закупка неакутальна,"[(закупка, PROD)]",[{'закупка': 'неакутальна'}]
2,"бк с фд, // есть потребность в лизинге и вкл 50 млн, как сдадут отчетность начнем сбор документов, депозиты нет. вэд нет, факторинг нет,","[(лизинге, PROD), (вкл, PROD), (депозиты, PROD), (вэд, PROD), (факторинг, PROD)]","[{'лизинге': '// есть потребность в и 50 млн'}, {'вкл': '// есть потребность в и 50 млн'}, {'депозиты': 'нет'}, {'вэд': 'нет'}, {'факторинг': 'нет'}]"
3,бк с гд. потребности в кредите.! лизинг нет потребности.,"[(кредите, PROD), (лизинг, PROD)]","[{'кредите': 'потребности в'}, {'лизинг': '! нет потребности'}]"
4,"кт, все также кредитование, депозиты, бг неинтересны, другие продукты тоже все выстроено со своим банком!","[(кредитование, PROD), (депозиты, PROD), (бг, PROD)]","[{'кредитование': 'все также'}, {'депозиты': ''}, {'бг': 'неинтересны'}]"
5,"кт с кд, обсудил по потребности, не нуждаются по кредитованию на тек момент. депозиты тоже не размещ сейчас. бг не польз. ввернуться просил к концу квартала!","[(кредитованию, PROD), (депозиты, PROD), (бг, PROD)]","[{'кредитованию': 'не нуждаются по на тек момент'}, {'депозиты': 'тоже не размещ сейчас'}, {'бг': 'не польз'}]"
6,"бк с гд\\, работают с сбер рсхб, кредит интересно вкл до 50 млн, депозиты нет, вэд да, юани 1млн, в китай, wm готов рассмотреть от 10 млн под 15,5%, созвон на сл неделе в среду по встрече","[(кредит, PROD), (вкл, PROD), (депозиты, PROD), (вэд, PROD)]","[{'кредит': 'интересно до 50 млн'}, {'вкл': 'интересно до 50 млн'}, {'депозиты': 'нет'}, {'вэд': 'да'}]"
7,проработать факторинг,"[(факторинг, PROD)]",[{'факторинг': 'проработать'}]
8,"бк с гд, пока свободных остатков нет, про меня помнит. сейчас ждет поступления и потом снова будем котировать. должно прийти 50 млн - 15,35% сейчас дала на 7 дней, условия интересны. по кредиту на февраль, приблизительно лими вкс 100+млн руб. точно скажет как будет понимание по планам на гд. жду ос","[(остатков, PROD), (кредиту, PROD)]","[{'остатков': 'пока свободных нет'}, {'кредиту': 'по на февраль'}]"
9,"кт с гб проговорили условия кредитония, интерес только на ип, должна направить инн, депозиты сейчас наврят ли ближе к марту. бг редко!","[(кредитония, PROD), (депозиты, PROD), (бг, PROD)]","[{'кредитония': 'кт с гб проговорили условия'}, {'депозиты': 'сейчас наврят ли ближе к марту'}, {'бг': 'редко !'}]"


In [ ]:
#ПЕРЕПИСЫВАЕМ ФУНКЦИЮ ИЗВЛЕЧЕНИЯ ГЛАВНОЙ МЫСЛИ
def extract_main_idea(sentence):
    main_idea = []
    for token in sentence:
        if not token.ent_type_:
            if token.dep_ == 'ROOT':
                main_idea.append(token.text)
            if token.head.dep_ == 'ROOT' and token.dep_ in ['XCOMP', 'NMOD', 'ADVMOD']:
                main_idea.append(token.text)
    return ' '.join(main_idea)

In [ ]:
train_data = [
    ('Мы редко ходим в кино', {'heads': [2, 2, 2, 0, 2], 'deps': ['nsubj', 'advmod', 'ROOT', 'prep', 'advmod']}),
    ('Депозиты не интересуют', {'heads': [3, 3, 0], 'deps': ['nsubj', 'neg', 'ROOT']})

]

In [ ]:
train_data = [
    ('договорились на встречу с фд на предмет проекн финансировакния', {'heads': [2, 2, 2, 0, 2], 'deps': ['nsubj', 'advmod', 'ROOT', 'prep', 'advmod']}),
    ('Депозиты не интересуют', {'heads': [3, 3, 0], 'deps': ['nsubj', 'neg', 'ROOT']})

]

In [ ]:
spacy.explain('pobj') #ОБЪЕКТ ПРЕДЛОГА, пошёл в лес
spacy.explain('prep') #ПРЕДЛОГ
spacy.explain('dobj') #ПРЯМОЙ ОБЪЕКТ, купил кофе
spacy.explain('cc') #КООРДИНИРУЮЩИЙ СОЮЗ
spacy.explain('neg')
advmod
xcomp = комплемент глагола
nmod = зависимое от главного слова существительное, предмет финансирования

TAG | MORPH | description
--- | --- | ---
ROOT | СКАЗУЕМОЕ |
NMOD | NOUN MODIFIER | зависимое от главного существительное слово ((в) альфе овер)
NSUBJ | NOMINAL SUBJECT | подлежащее (овердрафт устраивает, депозиты интересуют)
XCOMP | OPEN CLAUSAL COMPLEMENT| глагол / прилагательное, которое требует завершения фразы
OBL | OBLIQUE NOMINAL | обстоятельство ((в) лизинг интересует)
ADVMOD | ADVERBIAL MODIFIER | модификатор наречия, слово или фраза, которое уточняет значение наречия в предложении
CONJ | CONJUNCTION | союз
OBJ | OBJECT | существительно, на которое направлено действие
ADJ | ADJECTIVE | прилагательное

In [ ]:
jaccard_score = 0
for index, row in tqdm(data_to_train.iterrows(), total=data_to_train.shape[0]):
    text = row.text
    jaccard_score += jaccard(predict_entities(text, nlp), row.selected_text)

print(f'Average Jaccard Score is {jaccard_score / data_to_train.shape[0]}')

100%|██████████| 90/90 [00:02<00:00, 41.71it/s]

Average Jaccard Score is 0.35158730158730156


`.rfind` - это метод строки в Python, который выполняет обратный поиск подстроки в строке. Он возвращает **индекс** последнего вхождения подстроки. </br>
Если подстрока не найдена, возвращается -1 . </br>
В приведённых примерах метод `.rfind` используется для определения позиции знаков препинания, предшествующих упоминаниям сущностей.

```python
sentence_start = max(text.rfind('.', 0, token.idx), text.find(',', 0, token_idx), 0)
```
Использование `.rfind` позволяет определить позицию последнего вхождения знака препинания перед упоминанием сущности в тексте.

`.strip` - это метод в Python, который используется для удаления начальных (слева) и конечных (справа) пробелов или других указанных символов из строки.

In [ ]:
for index, row in comment_data.iterrows():
    text = row['comment']
    doc = nlp(text)
    for token in doc:
        for entity in ['овердрафт', 'бкл', 'гарантия', 'реф', 'депозит', 'рефить', 'лизинг']:
            if token.lemma_.lower() == entity:
                sentence_start = max(text.rfind('.', 0, token.idx), text.rfind(',', 0, token.idx), 0)
                min_nums = [text.find('.', token.idx, len(text)), text.find(',', token.idx, len(text)), len(text)]
                sentence_end = min(num for num in min_nums if num != -1)
                entity_text = text[sentence_start + 1 : sentence_end].strip()
                print(f'{entity}: {entity_text}')

#ПЕРЕХОДИМ К ИНСТРУМЕНТАМ SPACY, ТАК КАК СТАНДАРТНЫЕ ИНСТРУМЕНТЫ PYTHON ОБРЕЗАЮТ ТОКЕНЫ ВРОДЕ 9,8%

овердрафт: имеется овердрафт в ВТБ 40 млн руб под 9
реф: реф и вкл не интересует
овердрафт: полностью устраивает овердрафт
лизинг: кроме лизинга
лизинг: В лизинг интересует оборудование
рефить: рефить не хотят
депозит: Депозиты не интересуют


In [ ]:
def create_partlist(comment):
    partlist = []
    current_part = ''
    doc = nlp(comment)
    for token in doc:
        if (token.text == ',') or (token.text == '.'):
            partlist.append(current_part)
            current_part = ''
        else:
            current_part += token.text
            current_part += ' '
    if current_part:
        partlist.append(current_part)
    return partlist

https://www.kaggle.com/code/rohitsingh9990/ner-training-using-spacy-ensemble/notebook

https://www.kaggle.com/code/rohitsingh9990/ner-training-using-spacy-ensemble/comments

https://www.kaggle.com/competitions/tweet-sentiment-extraction/data?select=train.csv

Выделяем главную мысль из текста

```python
def extract_main_idea(sentence):
    main_idea = []
    for token in sentence:
        if token.dep_ not in ['punct', 'neg']:
            if token.dep_ == 'ROOT' or token.dep_ == 'conj':
                main_idea.append(token.text)
            elif token.dep_ == 'nsubj' and token.head.dep_ == 'ROOT':
                main_idea.append(token.text)
    return ' '.join(main_idea)

```
* aspect-based sentiment analysis
* spacy dependency parser: NER достанет аспект и все слова, которые к нему относятся
* spacy dependency parcer дообучать не нужно (это уже обученная модель и самому дообучить её не так просто), ей нужно пользоваться, чтобы извлекать морфологические зависимости в данных


Дообучение векторных представлений сводится к простой передаче текстовых данных для обновления векторных представлений. </br>
Обучение векторного пространства в spaCy не затрагивает independence parser и качество определения части речи напрямую. Векторное представление обучается с использованием нейронной сети, которая учится предсказать контекстное окружение слова в тексте. Это позволяет модели понимать семантические отношения между словами и использовать их для различные задачЮ такие как сравнение синонимов или классификация текста. </br>
Independence parser и определение части речи (Part-of-Speech tagging) в spaCy основаны на других компонентах модели, таких как морфологический анализатор и синтаксический анализатор. Эти компоненты модели обучаются отдельно и не зависят от обучения векторного пространства. </br>

```python
from spacy.util import compounding
from spacy.util import minibatch
```

```python
# Вычисляем косинусное расстояние между векторами
similarity = vector_cat.dot(vector_dog) / (vector_cat.norm() * vector_dog.norm())
```
```python
# Токенизируем и лемматизируем текст
processed_corpus = []
for doc in nlp.pipe(corpus):
    processed_corpus.append([token.lemma_ for token in doc])
```
Ниже пример функции, которая очищает текст от стоп-слов, пунктуации, лишних пробелов, чисел и возвращает лемматизированный текст.
```python
def lemmatize(doc):
    words = []
    for token in doc:
        if (token.is_stop != True) and (token.is_punct != True) and\
            (token.is_space != True) and (token.is_digit != True):
            words.append(token.lemma_)
    return ' '.join(words)
```
```python
def train_model(nlp, data, n_iter=10):
    for i in tange(n_iter):
        random.shuffle(data)
        losses = {}

        for batch in minibatch(data, size=compounding(4.0, 32.0, 1.001)):
            texts, - = zip(*batch)
            nlp.update(texts, losses = losses)

        print('Эпоха {}: Потери: {}'.format(i+1, losses))
```

```python
from spacy.training import Example

#ЗАМЕНИТЬ TRAIN_DATA НА РАЗМЕЧЕННЫЙ ДАТАСЕТ
for i in range(10):
    random.shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.5, losses=losses)
    print('Эпоха', i, ':', losses)
```



In [ ]:
data_to_train = pd.read_excel('train_data.xlsx') #ГАРАНТИЯ, СУЩНОСТЬ = 1 СЛОВО

# ФУНКЦИЯ ПРИВЕДЕНИЯ ДАННЫХ К ТИПУ ТЕКСТ - СУЩНОСТЬ
def get_train_data(data_to_train):
    train_data = []
    for index, row in data_to_train.iterrows():
        text = row.text
        selected_text = row.selected_text
        entity_list = []
        for elem in selected_text.split(sep = ','):
            elem = elem.strip()
            start = text.find(elem)
            end = start + len(elem)
            entity_list.append((start, end, 'PROD'))
        train_data.append((text, {'entities': entity_list}))
    return train_data

# ПОЛУЧЕНИЕ ДАННЫХ
train_data = get_train_data(data_to_train)

# ОБУЧЕНИЕ
for i in range(10):
    train_data = shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.1, losses=losses)
    print('Эпоха', i, ':', losses)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "ответил  гд евстигнеев вячеслав юрьевич, вопросы б..." with entities "[(117, 125, 'PROD'), (140, 142, 'PROD'), (144, 150...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "кт .кредитование, депозиты я не рассматривают от с..." with entities "[(4, 16, 'PROD'), (18, 26, 'PROD'), (63, 65, 'PROD...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Эпоха 0 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 5.983724868738863}
Эпоха 1 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 2.665724031962578}
Эпоха 2 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 6.75763171490603}
Эпоха 3 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 15.288516432095994}
Эпоха 4 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 4.1605849405589685}
Эпоха 5 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 7.308480607912801}
Эпоха 6 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 0.2053926237117818}
Эпоха 7 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 0.003934167754654847}
Эпоха 8 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 0.7260377700186291}
Эпоха 9 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 4.157964996430295}


In [ ]:
# МЕТРИКА ДЛЯ ИЗМЕРЕНИЯ КАЧЕСТВА NER - КОЭФ. ЖАККАРА
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def predict_entities(text, model):
    doc = model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    return selected_text

jaccard_score = 0
for index, row in tqdm(data_to_train.iterrows(), total=data_to_train.shape[0]):
    text = row.text
    jaccard_score += jaccard(predict_entities(text, nlp), row.selected_text)

print(f'Average Jaccard Score is {jaccard_score / data_to_train.shape[0]}')

  0%|          | 0/90 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/spacy/pipeline/attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
100%|██████████| 90/90 [00:02<00:00, 40.63it/s]

Average Jaccard Score is 0.37380952380952376


In [ ]:
# ОСТАВЛЯЕМ ТОЛЬКО СУЩНОСТИ И ИХ ОКРУЖЕНИЕ
def create_partlist(comment):
    partlist = []
    current_part = ''
    doc = nlp(comment)
    for token in doc:
        if (token.text == ',') or (token.text == '.'):
            partlist.append(current_part)
            current_part = ''
        else:
            current_part += token.text
            current_part += ' '
    if current_part:
        partlist.append(current_part)
    return partlist

def entity_extract(partlist):
    entlist = []
    for part in partlist:
        doc = nlp(part)
        for ent in doc.ents:
            if ent.label_ == 'PROD':
                entlist.append({ent.text: part}) #ПРИВОДИМ К ФОРМАТУ СУЩНОСТЬ-ЗНАЧЕНИЕ
    return entlist

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

data_to_train['partlist'] = data_to_train['text'].apply(create_partlist)
data_to_train['nlp'] = data_to_train['text'].apply(lambda x: nlp(x))
data_to_train['label'] = data_to_train['nlp'].apply(lambda x: [(ent.text, ent.label_) for ent in x.ents])
data_to_train['entlist'] = data_to_train['partlist'].apply(entity_extract)

data_to_train[['text', 'label', 'entlist']]

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
/usr/local/lib/python3.10/dist-packages/spacy/pipeline/attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)


,text,label,entlist
0,"кт с бухгалтером, предложение лизинга и кик, инфо просила направить на адрес (направлено), доведт до руководителя , созвонимся позже (гд в отъезде)!","[(лизинга, PROD), (кик, PROD)]","[{'лизинга': 'предложение лизинга и кик '}, {'кик': 'предложение лизинга и кик '}]"
1,закупка неакутальна,"[(закупка, PROD)]",[{'закупка': 'закупка неакутальна '}]
2,"бк с фд, // есть потребность в лизинге и вкл 50 млн, как сдадут отчетность начнем сбор документов, депозиты нет. вэд нет, факторинг нет,","[(лизинге, PROD), (вкл, PROD), (депозиты, PROD), (вэд, PROD), (факторинг, PROD)]","[{'лизинге': '// есть потребность в лизинге и вкл 50 млн '}, {'вкл': '// есть потребность в лизинге и вкл 50 млн '}, {'депозиты': 'депозиты нет '}, {'вэд': 'вэд нет '}, {'факторинг': 'факторинг нет '}]"
3,бк с гд. потребности в кредите.! лизинг нет потребности.,"[(кредите, PROD), (лизинг, PROD)]","[{'кредите': 'потребности в кредите '}, {'лизинг': '! лизинг нет потребности '}]"
4,"кт, все также кредитование, депозиты, бг неинтересны, другие продукты тоже все выстроено со своим банком!","[(кредитование, PROD), (депозиты, PROD), (бг, PROD)]","[{'кредитование': 'все также кредитование '}, {'депозиты': 'депозиты '}, {'бг': 'бг неинтересны '}]"
5,"кт с кд, обсудил по потребности, не нуждаются по кредитованию на тек момент. депозиты тоже не размещ сейчас. бг не польз. ввернуться просил к концу квартала!","[(кредитованию, PROD), (депозиты, PROD), (бг, PROD)]","[{'кредитованию': 'не нуждаются по кредитованию на тек момент '}, {'депозиты': 'депозиты тоже не размещ сейчас '}, {'бг': 'бг не польз '}]"
6,"бк с гд\\, работают с сбер рсхб, кредит интересно вкл до 50 млн, депозиты нет, вэд да, юани 1млн, в китай, wm готов рассмотреть от 10 млн под 15,5%, созвон на сл неделе в среду по встрече","[(кредит, PROD), (вкл, PROD), (депозиты, PROD), (вэд, PROD)]","[{'кредит': 'кредит интересно вкл до 50 млн '}, {'вкл': 'кредит интересно вкл до 50 млн '}, {'депозиты': 'депозиты нет '}, {'вэд': 'вэд да '}]"
7,проработать факторинг,"[(факторинг, PROD)]",[{'факторинг': 'проработать факторинг '}]
8,"бк с гд, пока свободных остатков нет, про меня помнит. сейчас ждет поступления и потом снова будем котировать. должно прийти 50 млн - 15,35% сейчас дала на 7 дней, условия интересны. по кредиту на февраль, приблизительно лими вкс 100+млн руб. точно скажет как будет понимание по планам на гд. жду ос","[(остатков, PROD), (кредиту, PROD)]","[{'остатков': 'пока свободных остатков нет '}, {'кредиту': 'по кредиту на февраль '}]"
9,"кт с гб проговорили условия кредитония, интерес только на ип, должна направить инн, депозиты сейчас наврят ли ближе к марту. бг редко!","[(кредитония, PROD), (депозиты, PROD), (бг, PROD)]","[{'кредитония': 'кт с гб проговорили условия кредитония '}, {'депозиты': 'депозиты сейчас наврят ли ближе к марту '}, {'бг': 'бг редко ! '}]"


In [ ]:
# HEATMAP' S FROM SEABOOOORN